# Day 3

Full text of the task can be found on [AoC website](https://adventofcode.com/2023/day/3).

We arrived at the gondola lift that will take us up to the water source. But the gondolas don't work - they are missing a part! 

Another ChatGPT 4 & DALL·E illustration:

<img src="./ChatGPT_illustrations/day03.png" width="400" />

## Part One

We need to sum up the part numbers of all engine parts. The parts are marked by an adjacent symbol. 

This sounds tricky. Given a grid schematic, we need to figure out which of the numbers are adjacent to a symbol (in all directions). The dot (`.`) is not a symbol. Those number will be used for the sum.

|   | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9  |
|---|---|---|---|---|---|---|---|---|---|----|
| 0 | 4 | 6 | 7 | . | . | **1** | **1** | **4** | . | .  |
| 1 | . | . | . | * | . | . | . | . | . | .  |
| 2 | . | . | 3 | 5 | . | . | 6 | 3 | 3 | .  |
| 3 | . | . | . | . | . | . | # | . | . | .  |
| 4 | 6 | 1 | 7 | * | . | . | . | . | . | .  |
| 5 | . | . | . | . | . | + | . | **5** | **8** | .  |
| 6 | . | . | 5 | 9 | 2 | . | . | . | . | .  |
| 7 | . | . | . | . | . | 7 | 5 | 5 | . | .  |
| 8 | . | . | . | $ | . | * | . | . | . | .  |
| 9 | . | 6 | 6 | 4 | . | 5 | 9 | 8 | . | .  |


In this example only `114` in the zero line and `58` in the 5th line are not adjacent to a symbol. The sum of all other numbers is equal to `4361`.


Looks like I need to get the position of each number and then check the position around it. 

In [1]:
example_input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""


In [2]:
from typing import Tuple, Set, Dict
def process_input(input_file: str, 
                  example: bool = False) -> Tuple[Set[Tuple[int, int]], 
                                                  Dict[Tuple[int, int], int]]:
    """
    Process the input file and return symbols and numbers dictionary.

    Parameters:
    - input_file (str): Path to the input file or the input data itself.
    - example (bool): Flag indicating whether the input is an example or a file path.

    Returns:
    - symbols (set): Set of symbols.
    - numbers_dict (dict): Dictionary of numbers with their coordinates as keys.

    """
    symbols = set()
    numbers_dict = {}
    y = 0

    if not example:
        with open(input_file) as f:
            input_data = f.read()
    else:
        input_data = input_file

    for line in input_data.splitlines():
        line += "."  # Add sentinel to process last number
        num = ""
        for x, c in enumerate(line):
            if c.isdigit():
                num += c
            else:
                if num:
                    numbers_dict[(x-len(num), y)] = int(num)
                    num = ""
                if c != '.':
                    symbols.add((x, y))
        y += 1

    return symbols, numbers_dict

process_input(example_input, True)

({(3, 1), (3, 4), (3, 8), (5, 5), (5, 8), (6, 3)},
 {(0, 0): 467,
  (5, 0): 114,
  (2, 2): 35,
  (6, 2): 633,
  (0, 4): 617,
  (7, 5): 58,
  (2, 6): 592,
  (6, 7): 755,
  (1, 9): 664,
  (5, 9): 598})

In [3]:
def scan_grid(symbols: set, numbers_dict: dict) -> int:
    """
    Scan the grid for parts numbers. Those are defined as numbers that are
    adjacent to a symbol. The sum of all parts numbers is the solution.
    
    Parameters:
        symbols: Set of coordinates of symbols.
        numbers_dict: Dictionary of coordinates of numbers and their value.
    
    Returns:
        Sum of all parts numbers.
    """
    parts_sum = 0
    for (x, y), num in numbers_dict.items():
        num_digits = len(str(num))
        x_min = x - 1
        x_max = x + num_digits
        xs = [x1 for x1 in range(x_min, x_max+1) if x1 >= 0]
        ys = [y1 for y1 in [y-1, y+1] if y1 >= 0]
        pos = [(x1, y1) for x1 in xs for y1 in ys]
        pos += [(x1, y) for x1 in [x_min, x_max] if x1 >= 0]
        if any(p in symbols for p in pos):
            parts_sum += num

    return parts_sum

scan_grid(*process_input(example_input, True))

4361

I needed debug for a bit bc I forgot to add numbers at the end of the line in  `process_input`. 

In [4]:
def print_matrix(input_string: str) -> None:
    """
    Prints a matrix representation of the input string.

    Args:
        input_string (str): The input string representing the matrix.

    Returns:
        None
    """
    # Split the input string into rows
    rows = input_string.split("\n")
    # get the max length of the rows
    max_len = max(len(row) for row in rows)
    
    # Create a matrix of the max length
    matrix = [[char for char in row.ljust(max_len)] for row in rows]

    # Print the matrix with row and column numbers
    print(f"   | {'| '.join(str(i).ljust(2) for i in range(max_len))}|")
    print(f" {'-'*(max_len*4 + 4)}")
    for i, row in enumerate(matrix):
        print(f"{str(i).ljust(2)} | {' | '.join(row)} |")

# Input string
input_string = """....
...1
989*
._--"""

# Print the matrix
print_matrix(input_string)


   | 0 | 1 | 2 | 3 |
 --------------------
0  | . | . | . | . |
1  | . | . | . | 1 |
2  | 9 | 8 | 9 | * |
3  | . | _ | - | - |


In [5]:
corner_case = """111$12
+.....
..13.%"""

print_matrix(corner_case)

assert(scan_grid(*process_input(corner_case, True)) == 123)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
0  | 1 | 1 | 1 | $ | 1 | 2 |
1  | + | . | . | . | . | . |
2  | . | . | 1 | 3 | . | % |


In [6]:
corner_case = """111$12
+...8.
..13.%"""

print_matrix(corner_case)

assert(scan_grid(*process_input(corner_case, True)) == 131)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
0  | 1 | 1 | 1 | $ | 1 | 2 |
1  | + | . | . | . | 8 | . |
2  | . | . | 1 | 3 | . | % |


In [7]:
corner_case = """..$$..
+...8.
..13.%"""

print_matrix(corner_case)

assert(scan_grid(*process_input(corner_case, True)) == 8)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
0  | . | . | $ | $ | . | . |
1  | + | . | . | . | 8 | . |
2  | . | . | 1 | 3 | . | % |


In [8]:
corner_case = """1.3
.5.
7.9"""

print_matrix(corner_case)

assert(scan_grid(*process_input(corner_case, True)) == 0)

   | 0 | 1 | 2 |
 ----------------
0  | 1 | . | 3 |
1  | . | 5 | . |
2  | 7 | . | 9 |


In [9]:
corner_case = """1*3
*5*
7*9"""

print_matrix(corner_case)

assert(scan_grid(*process_input(corner_case, True)) == 25)

   | 0 | 1 | 2 |
 ----------------
0  | 1 | * | 3 |
1  | * | 5 | * |
2  | 7 | * | 9 |


In [10]:
corner_case = """.$12"""
print_matrix(corner_case)
assert(scan_grid(*process_input(corner_case, True)) == 12)

   | 0 | 1 | 2 | 3 |
 --------------------
0  | . | $ | 1 | 2 |


In [11]:
corner_case = """***"""
print_matrix(corner_case)
assert(scan_grid(*process_input(corner_case, True)) == 0)

   | 0 | 1 | 2 |
 ----------------
0  | * | * | * |


In [12]:
corner_case_from_reddit = """333.3
...*."""
print_matrix(corner_case_from_reddit)
assert(scan_grid(*process_input(corner_case_from_reddit, True)) == 336)

   | 0 | 1 | 2 | 3 | 4 |
 ------------------------
0  | 3 | 3 | 3 | . | 3 |
1  | . | . | . | * | . |


In [13]:
corner_case = """1.3
.5*
..9"""
print_matrix(corner_case)
assert(scan_grid(*process_input(corner_case, True)) == 17)

   | 0 | 1 | 2 |
 ----------------
0  | 1 | . | 3 |
1  | . | 5 | * |
2  | . | . | 9 |


In [14]:
corner_case = """*.*
123"""
print_matrix(corner_case)
assert(scan_grid(*process_input(corner_case, True)) == 123)

   | 0 | 1 | 2 |
 ----------------
0  | * | . | * |
1  | 1 | 2 | 3 |


In [15]:
scan_grid(*process_input('./inputs/day03.txt'))

539637

That's the right answer! You are one gold star ⭐ closer to restoring snow operations.

## Part Two

In this part, we are actually only interested in '*' near two numbers. Then we calculate the gear ratio by multiplying those numbers and adding them to the sum.

|   | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9  |
|---|---|---|---|---|---|---|---|---|---|----|
| 0 | **4** | **6** | **7** | . | . | 1 | 1 | 4 | . | .  |
| 1 | . | . | . | * | . | . | . | . | . | .  |
| 2 | . | . | **3** | **5** | . | . | 6 | 3 | 3 | .  |
| 3 | . | . | . | . | . | . | # | . | . | .  |
| 4 | 6 | 1 | 7 | * | . | . | . | . | . | .  |
| 5 | . | . | . | . | . | + | . | 5 | 8 | .  |
| 6 | . | . | 5 | 9 | 2 | . | . | . | . | .  |
| 7 | . | . | . | . | . | **7** | **5** | **5** | . | .  |
| 8 | . | . | . | $ | . | * | . | . | . | .  |
| 9 | . | 6 | 6 | 4 | . | **5** | **9** | **8** | . | .  |

There are two gears here - `467` and `35` with gear ratio `16345` and `755` and `598` with gear ratio `451490`. Together, the sum is `467835`.

In [16]:
def process_input_for_gears(input_file: str, 
                            example: bool = False) -> Tuple[Set[Tuple[int, int]],
                                                            Dict[int, str],
                                                            Dict[Tuple[int, int], int]]:
    """
    Process the input file and extract gears, numbers_idx, and numbers_dict.

    Parameters:
        input_file (str): The path to the input file.
        example (bool): Flag indicating whether the input is an example or not.

    Returns:
        tuple: A tuple containing gears (set), numbers_idx (dict), and numbers_dict (dict).
    """
    gears = set()
    numbers_idx = dict()
    numbers_dict = dict()
    y = 0
    idx = 0
    if not example:
        with open(input_file) as f:
            input_data = f.read()
    else:
        input_data = input_file

    for line in input_data.splitlines():
        line += "."  # Add sentinel to process last number
        num = ""
        for x, c in enumerate(line):
            if c.isdigit():
                num += c
            else:
                if num:
                    numbers_idx[idx] = num
                    for i in range(len(num)):
                        numbers_dict[(x - i - 1, y)] = idx
                    num = ""
                    idx += 1
                if c == '*':
                    gears.add((x, y))
        y += 1

    return gears, numbers_idx, numbers_dict


In [17]:
def scan_grid_for_gears(gears: set, 
                        numbers_idx: dict, 
                        numbers_dict: dict) -> int:
    """
    Scans the grid for gears and calculates the sum of the products of the corresponding numbers.

    Parameters:
        gears (set): List of coordinates representing the positions of gears.
        numbers_idx (dict): Dictionary mapping gear indices to numbers.
        numbers_dict (dict): Dictionary mapping positions to gear indices.

    Returns:
        gears_sum (int): Sum of the products of the corresponding numbers.

    """
    gears_sum = 0
    for (x, y) in gears:
        xs = [x1 for x1 in [x-1, x, x+1] if x1 >= 0]
        ys = [y1 for y1 in [y-1, y, y+1] if y1 >= 0]
        pos = [(x1, y1) for x1 in xs for y1 in ys if (x1, y1) != (x, y)]
        idxs = {numbers_dict.get(p) for p in pos if numbers_dict.get(p) is not None}
        if len(idxs) == 2:
            nums = list(map(int, [numbers_idx.get(i) for i in idxs]))
            gears_sum += nums[0] * nums[1]

    return gears_sum

assert(scan_grid_for_gears(*process_input_for_gears(example_input, True)) == 467835)

In [18]:
corner_case = """1.3
.5*
..9"""
print_matrix(corner_case)
assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 0)

   | 0 | 1 | 2 |
 ----------------
0  | 1 | . | 3 |
1  | . | 5 | * |
2  | . | . | 9 |


In [19]:
corner_case = """*.*
123"""
print_matrix(corner_case)
assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 0)

   | 0 | 1 | 2 |
 ----------------
0  | * | . | * |
1  | 1 | 2 | 3 |


In [20]:
corner_case = """.*.
1.3"""
print_matrix(corner_case)
assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 3)

   | 0 | 1 | 2 |
 ----------------
0  | . | * | . |
1  | 1 | . | 3 |


In [21]:
corner_case = """***"""
print_matrix(corner_case)
assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 0)

   | 0 | 1 | 2 |
 ----------------
0  | * | * | * |


In [22]:
corner_case = """111*12
*...8.
..13.*"""

print_matrix(corner_case)

assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 0)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
0  | 1 | 1 | 1 | * | 1 | 2 |
1  | * | . | . | . | 8 | . |
2  | . | . | 1 | 3 | . | * |


In [23]:
corner_case = """111*12
*...8.
..13**"""

print_matrix(corner_case)

assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 8*13)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
0  | 1 | 1 | 1 | * | 1 | 2 |
1  | * | . | . | . | 8 | . |
2  | . | . | 1 | 3 | * | * |


In [24]:
corner_case = """111*12
*.*.8.
..13**"""

print_matrix(corner_case)

assert(scan_grid_for_gears(*process_input_for_gears(corner_case, True)) == 8*13+111*13)

   | 0 | 1 | 2 | 3 | 4 | 5 |
 ----------------------------
0  | 1 | 1 | 1 | * | 1 | 2 |
1  | * | . | * | . | 8 | . |
2  | . | . | 1 | 3 | * | * |


In [25]:
scan_grid_for_gears(*process_input_for_gears('./inputs/day03.txt'))

82818007

That's the right answer! You are one gold star ⭐ closer to restoring snow operations.